# Data Gathering and Cleaning

Data for this project is gathered from the U.S. Energy Information Administration (EIA) [website](https://www.eia.gov).

## Import packages

In [146]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import requests
import time
from bs4 import BeautifulSoup as BS

%load_ext autoreload
%autoreload 2
import helper_functions


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We have to do a little manipulation to the text file from EIA. It is a text file with a bunch of line-separated JSON objects, but I massage it here to a proper JSON and export it as a new file.

In [24]:
lastline = None

# Open and read text file
with open("SEDS.txt","r") as f:
    lineList = f.readlines()
    
    # Keep track of last line
    lastline=lineList[-1]

# Open text file and create new json to be written
with open("SEDS.txt","r") as f, open("cleanfile.json","w") as g:
    
    # Iterate through each line of the text file
    for i,line in enumerate(f,0):
        
        # First line gets [ and , to initialize the json
        if i == 0:
            line = "["+str(line)+","
            g.write(line)
            
        # Last line gets ] to signal the end of the json
        elif line == lastline:            
            g.write(line)
            g.write("]")
            
        # Other lines get comma separation
        else:
            line = str(line)+","
            g.write(line)

In [33]:
file = open('cleanfile.json', 'r')
json_data = json.load(file)

The following energy types were selected based on the categories in the [EIA educational page](https://www.eia.gov/energyexplained/energy-and-the-environment/).

In [29]:
energy_types = [
    'All Petroleum Products excluding Fuel Ethanol',
    'Coal',
    'Natural Gas including Supplemental Gaseous Fuels',
    'Nuclear Power',
    'Biomass',
    'Fuel Ethanol excluding Denaturant',
    'Geothermal',
    'Hydroelectricity',
    'Solar Energy',
    'Wind Energy',
    'Renewable Energy'
]



# Make all lowercase in case some pages have inconsistent letter casing
for i in range(len(energy_types)):
    energy_types[i] = energy_types[i].lower()
    
nonrenewable_energies = energy_types[:4]

renewable_energies = energy_types[4:]

## Web scrape to get the information for each of the above types of energy

In [34]:
headers = {'user-agent': 'Safari/13.0.2 (Macintosh; Intel Mac OS X 10_15)'}
base_url = 'https://www.eia.gov/opendata/qb.php'
consumption_suffix = '?category=40204'

In [35]:
consumption_page = helper_functions.get_page(base_url+consumption_suffix,headers)

In [36]:
# Create empty dict to store all info across every sector and energy type by state
env_series_ids = {}

# Start by scraping the consumption website in order to get the list of available sectors  
consumption_sectors = consumption_page.find('div',{'class':'pagecontent mr_temp2'})

# Store sector url suffixes in a list
sector_url_suffixes = [sector.a['href'] for sector in consumption_sectors.find_all('li')[:7]]

# Loop 1 - iterate through each sector
for sector_url_suffix in sector_url_suffixes:
    
    # Scrape the sector page
    sector_page = helper_functions.get_page(base_url+sector_url_suffix,headers)    

    # Go into first url and grab tags of all children categories
    children_categories = sector_page.find('section').ul.find_all('li')

    # Store the urls of children cats (ccats = children categories)
    ccats_url_suffixes = [children_category.a['href'] 
                          for children_category in children_categories
                          if children_category.text.lower() in energy_types]
    
    # Loop 2 - for each sector, iterate through the relevant types of energy consumption to get state-level data
    for ccats_url_suffix in ccats_url_suffixes:
        
        # Scrape the child category page
        child_category_page = helper_functions.get_page(base_url+ccats_url_suffix,headers)

        # Grab tags of all energy unit children categories. Only want Btu
        energy_unit_cats = child_category_page.find('div',{'class':'main_col'}).ul.find_all('li')

        # Store only the url of the 'Btu' children category. I make a list and select only the first element 
        # because sometimes there will be two energy unit options or just one. This way ensures we only take 
        # the Btu option.
        btu_url_suffix = [energy_unit.a['href'] 
                   for energy_unit in energy_unit_cats
                   if energy_unit.text == 'Btu'][0]
        
        # Scrape the Btu page
        btu_page = helper_functions.get_page(base_url+btu_url_suffix,headers)
        
        # Get list of states by their tags
        states = btu_page.find('div',{'class':'main_col'}).ul.find_all('li')
        
        # Get url suffixes for each state
        state_url_suffixes = [state.a['href'] for state in states]
        
        # Isolate the sector and energy type
        sector = btu_page.find('div',{'class':'main_col'}).h3.find_all('a')[3].text
        energy_type = btu_page.find('div',{'class':'main_col'}).h3.find_all('a')[4].text
        
        # Add these to a dict which will be the values of the overarching env_series_ids dict
        series_id_values = {'sector':sector,'energy_type':energy_type}
        
        # Parse through url suffixes to get and store the series ids we want to use to parse the big JSON
        for state_suffix in state_url_suffixes:
            series_id = re.findall('SEDS.*',state_suffix)[0]
            env_series_ids[series_id] = series_id_values
            

### Parse energy data

In [37]:
# Set up empty bucket for parsed data
environmental_data = []

# Iterate through big json to parse relevant info
for single_json in json_data:
    
    # Only parse entries that have the series ids that we care about
    if single_json.get('series_id') in env_series_ids.keys():
        single_data_entry = {}
        single_data_entry['series_id'] = single_json['series_id']
        single_data_entry['sector'] = env_series_ids[single_json['series_id']]['sector']
        single_data_entry['data'] = single_json['data']
        single_data_entry['state'] = re.findall(', .*$',single_json['name'])[0][2:]
        single_data_entry['units'] = single_json['units']
        single_data_entry['energy_type'] = env_series_ids[single_json['series_id']]['energy_type']
        # Add a column for whether or not this type of energy is renewable
        if single_data_entry['energy_type'] in renewable_energies:
            single_data_entry['renewable'] = 1
        else: 
            single_data_entry['renewable'] = 0
        environmental_data.append(single_data_entry)

## Web scrape for population and GDP data

In [47]:
population_url_suffix = '?category=40367'
gdp_url_suffix = '?category=40828'
pop_gdp_url_suffixes = [population_url_suffix, gdp_url_suffix]

# Create container for  data
pop_gdp_series_ids = {}

for pop_gdp_url_suffix in pop_gdp_url_suffixes:
    
    # Scrape population page
    page = helper_functions.get_page(base_url + pop_gdp_url_suffix,headers)

    # Isolate html tags containing urls for each state
    state_tags = page.find('div',{'class':'main_col'}).ul.find_all('li')

    # Extract and save each state url suffix
    state_url_suffixes = [state_tag.a['href'] 
                          for state_tag in state_tags]

    # Iterate through each state url suffix to extract features
    for state_url_suffix in state_url_suffixes:

        # Scrape each state's series page
        state_page = helper_functions.get_page(base_url + state_url_suffix,headers)

        # Isolate html tags containing state name, get text from tag, parse for name
        api_call_tags = state_page.find('div',{'class':'main_col'}).find('div',{'class':'api_call_container'})
        state_text = api_call_tags.find_all('p')[1].text
        state = re.findall('(, )(.*)',state_text)[0][1]
        
        # Isolate html tags containing description (gdp or pop), get text from tag
        main_col_tags = state_page.find('div',{'class':'main_col'}).h3
        desc = main_col_tags.find_all('a')[2].text
        
        # Parse url suffix for series id
        series_id = re.findall('SEDS.*',state_url_suffix)[0]

        # Add to data container
        values = {'state':state,'description':desc}
        pop_gdp_series_ids[series_id] = values

### Parse population and gdp data

In [48]:
# Set up empty bucket for parsed data
pop_gdp_data = []

# Iterate through big json to parse relevant info
for single_json in json_data:
    
    # Only parse entries that have the series ids that we care about
    if single_json.get('series_id') in pop_gdp_series_ids.keys():
        single_data_entry = {}
        single_data_entry['series_id'] = single_json['series_id']
        single_data_entry['description'] = pop_gdp_series_ids[single_json['series_id']]['description']
        single_data_entry['units'] = single_json['units']
        single_data_entry['data'] = single_json['data']
        single_data_entry['state'] = pop_gdp_series_ids[single_json['series_id']]['state']
        pop_gdp_data.append(single_data_entry)

# Get Weather data from NCEI 
National Centeres for Environmental Information has an Access Data Service that provides a RESTful API.

In [59]:
# List of GHCND station codes
with open("ghcnd.txt","r") as f:
    lineList = f.readlines()

# Turn each line into a list for easier sorting
split_lines = [line.split() for line in lineList]

# The city names got split too, so I rejoin them back together
for i in range(len(split_lines)):
    if len(split_lines[i]) > 6:
        split_lines[i] = split_lines[i][:5] + [' '.join(split_lines[i][5:])]

In [114]:
# Import the state capitals, convert to dict, and rename a couple of them
import csv
state_capitals = open('us-state-capitals.csv')
state_capitals_reader = csv.reader(state_capitals)
state_capitals_dict = dict(state_capitals_reader)
state_capitals_dict['CA'] = 'FRESNO'      # Sacramento is not in the .txt file so i picked city near the center of the state
state_capitals_dict['MN'] = 'ST PAUL'     # Abbreviate saint
state_capitals_dict['OR'] = 'BEND'        # Salem is not in the .txt file so i picked city near the center of the state
state_capitals_dict['SD'] = 'FT PIERRE'   # Abbreviate fort

# Make all uppercase to be able to match to the .txt file
for key, value in state_capitals_dict.items():
    state_capitals_dict[key] = state_capitals_dict[key].upper()

In [119]:
# Empty container to add stations information to
weather_stations = []

# Iterate through each state/city (key/value) pair
for key, value in state_capitals_dict.items():
    
    # For each state/city pair, look through .txt file
    for line in split_lines:
        
        # store relevant station ids, states, and cities
        if key == line[4] and re.findall(value+'.*',line[5]):
            weather_stations.append({'station':line[0],'state':line[4],'city':line[5]})
            break

In [142]:
weather_stations[0]['station'] = 

{'station': 'USC00015547', 'state': 'AL', 'city': 'MONTGOMERY #2'}

In [140]:
base_url = 'https://www.ncei.noaa.gov/access/services/data/v1'

params = {'dataset': 'daily-summaries',   # the dataset to query for data
          'stations': 'USC00015547',      # comma separated list of station identifiers for selection and subsetting
          'startDate': '2000-01-01',      # YYYY-MM-DD
          'endDate' : '2000-12-31',       # YYYY-MM-DD
          'format' : 'json',              # json is ideal format   
          'includeAttributes': 'True',
          'includeStationName' : 'True',  # Just to be sure we're looking at the right city/state
          'units' : 'standard',           # degrees fahrenheit
         }        

weather_list = requests.get(base_url, params = params).json()

In [141]:
weather_list

[{'DATE': '2000-01-01',
  'STATION': 'USC00015547',
  'SNOW': '0.0',
  'PRCP_ATTRIBUTES': ',,0,0600',
  'SNOW_ATTRIBUTES': 'P,,0',
  'PRCP': '0.00',
  'SNWD_ATTRIBUTES': 'P,,0',
  'SNWD': '0.0',
  'NAME': 'MONTGOMERY NUMBER 2, AL US'},
 {'DATE': '2000-01-02',
  'STATION': 'USC00015547',
  'SNOW': '0.0',
  'PRCP_ATTRIBUTES': ',,0,0600',
  'SNOW_ATTRIBUTES': 'P,,0',
  'PRCP': '0.00',
  'SNWD_ATTRIBUTES': 'P,,0',
  'SNWD': '0.0',
  'NAME': 'MONTGOMERY NUMBER 2, AL US'},
 {'DATE': '2000-01-03',
  'STATION': 'USC00015547',
  'SNOW': '0.0',
  'PRCP_ATTRIBUTES': ',,0,0600',
  'SNOW_ATTRIBUTES': 'P,,0',
  'PRCP': '0.00',
  'SNWD_ATTRIBUTES': 'P,,0',
  'SNWD': '0.0',
  'NAME': 'MONTGOMERY NUMBER 2, AL US'},
 {'DATE': '2000-01-04',
  'STATION': 'USC00015547',
  'SNOW': '0.0',
  'PRCP_ATTRIBUTES': ',,0,0600',
  'SNOW_ATTRIBUTES': 'P,,0',
  'PRCP': '0.59',
  'SNWD_ATTRIBUTES': 'P,,0',
  'SNWD': '0.0',
  'NAME': 'MONTGOMERY NUMBER 2, AL US'},
 {'DATE': '2000-01-05',
  'STATION': 'USC00015547',
  'S

In [135]:
weather_list[0]

new_dict = {}
new_dict['date'] = element['DATE']
new_dict['state'] = re.findall('(, )(.{2})',element['NAME'])[0][1]


{'DATE': '2000-01-01',
 'STATION': 'USC00015547',
 'SNOW': '0.0',
 'PRCP': '0.00',
 'SNWD': '0.0',
 'NAME': 'MONTGOMERY NUMBER 2, AL US'}

In [160]:
def get_station_weather(station):
    """
    Returns: A parsed json of the weather data for the given station from 1960 - 2018.
    
    Param station: [str] A station code from the list of stations
    """

    base_url = 'https://www.ncei.noaa.gov/access/services/data/v1'

    params = {'dataset': 'daily-summaries',   # the dataset to query for data
              'stations': f'{station}',      # comma separated list of station identifiers for selection and subsetting
              'startDate': '2000-01-01',      # YYYY-MM-DD
              'endDate' : '2000-01-01',       # YYYY-MM-DD
              'format' : 'json',              # json is ideal formal    
              'includeStationName' : 'True',  # Just to be sure we're looking at the right city/state
              'units' : 'standard',           # degrees fahrenheit
             }        

    return requests.get(base_url, params = params).json()

In [172]:
weather_stations[0]['station'] = 'USC00015553'
weather_stations[0]['city'] = 'MONTGOMERY WB CITY'

weather_stations[1]['station'] = 'USC00504092'
weather_stations[1]['city'] = 'JUNEAU LENA PT'

weather_stations[2]['station'] = 'USC00026513'
weather_stations[2]['city'] = 'PICACHO 8 SE'

weather_stations[3]['station'] = 'USC00034562'
weather_stations[3]['city'] = 'MALVERN'

weather_stations[4]['station'] = 'USC00043261'
weather_stations[4]['city'] = 'FRIANT GOVERNMENT CAMP'

weather_stations[5]['station'] = 'USC00052223'
weather_stations[5]['city'] = 'DENVER WATER DEPT'

weather_stations[6]['station'] = 'USC00063207'
weather_stations[6]['city'] = 'GROTON'

weather_stations[8]['station'] = 'USC00084802'
weather_stations[8]['city'] = 'LAKELAND 2'

In [222]:
weather_stations[9]

{'station': 'USC00090444', 'state': 'GA', 'city': 'ATLANTA 9NW'}

In [161]:
station_data = []
for station in weather_stations:
    station_data.append(get_station_weather(station['station']))
    time.sleep(0.1)
    

In [229]:
get_station_weather('USC00095447')

[]

In [200]:
station_data

[[{'DATE': '2000-01-01',
   'STATION': 'USC00015547',
   'SNOW': '0.0',
   'PRCP': '0.00',
   'SNWD': '0.0',
   'NAME': 'MONTGOMERY NUMBER 2, AL US'}],
 [],
 [],
 [],
 [],
 [],
 [],
 [{'DATE': '2000-01-01',
   'STATION': 'USC00072730',
   'SNOW': '0.0',
   'WT01': '    1',
   'TMAX': '48',
   'TMIN': '28',
   'PRCP': '0.00',
   'TOBS': '43',
   'SNWD': '0.0',
   'NAME': 'DOVER, DE US'}],
 [],
 [{'DATE': '2000-01-01',
   'STATION': 'USC00090444',
   'SNOW': '0.0',
   'PRCP': '0.00',
   'SNWD': '0.0',
   'NAME': 'ATLANTA 9 NW, GA US'}],
 [{'EVAP': '0.11',
   'STATION': 'USC00511918',
   'SNOW': '0.0',
   'PRCP': '0.00',
   'TOBS': '66',
   'MXPN': '85',
   'SNWD': '0.0',
   'NAME': 'HONOLULU OBSERVATORY 702.2, HI US',
   'DATE': '2000-01-01',
   'TMAX': '79',
   'TMIN': '61',
   'MNPN': '70',
   'WDMV': '37.3'}],
 [{'DATE': '2000-01-01',
   'STATION': 'USC00101017',
   'SNOW': '0.2',
   'WT01': '    1',
   'TMAX': '31',
   'TMIN': '19',
   'PRCP': '0.01',
   'TOBS': '26',
   'SNWD': '0.0

In [198]:
station_data[1]

[]

## Storing data into MongoDB

In [14]:
import pymongo
from pprint import pprint

client = pymongo.MongoClient('mongodb://localhost/')
db = client.admin

# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)

mydb = client['energy_data']

print(client.list_database_names())

{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 12, 'warning': 0},
 'connections': {'active': 1,
                 'available': 3271,
                 'current': 5,
                 'totalCreated': 10},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCatchUpsAlreadyCaughtUp': 0,
                     'numCatchUpsFailedWithError': 0,
                     'numCatchUpsFailedWithNewTerm': 0,
                     'numCatchUpsFailedWithReplSetAbortPrimaryCatchUpCmd': 0,
                     'numCatchUpsSkipped': 0,
                     'numCatchUpsSucceeded': 0,
                     'numCatchUpsTimedOut': 0,
                     'numStepDownsCausedByHigherTerm': 0,
                     'priorityTakeover': {'c

In [16]:
energy_collection = mydb['energy_data']

In [45]:
env_results = energy_collection.insert_many(environmental_data)

In [49]:
pop_gdp_results = energy_collection.insert_many(pop_gdp_data)

### Practice querying

In [35]:
all_states = [x for x in energy_collection.find({})]

In [41]:
count = 0
gdp_count = 0
for series in all_states:
    if len(series.get('data')) < 58:
        count+=1
        if series.get('description') == 'GDP':
            gdp_count += 1
        
        
        
#         print(series)
    
print(f'there are {count} series with less than 58 data points')
print(f'of those, {gdp_count} of them are gdp')

there are 52 series with less than 58 data points
of those, 52 of them are gdp


Did some digging and turns out GDP only goes back to 1997 because that was when data transitioned from Standard Industrial Classification (SIC) to North American Industry Classification System (NAICS). This gives us only about 20 years of data to work with for GDP. Since our data is in annual increments, we only have roughly 20 data points which will not be enough for time series analysis (usually sounds like 50-60 points minimum). Thus, we will exclude GDP from the exogenous vars.

In [29]:
short_list['data']

[['2017', 2175],
 ['2016', 1983],
 ['2015', 1787],
 ['2014', 1679],
 ['2013', 1562],
 ['2012', 1475],
 ['2011', 1377],
 ['2010', 1282],
 ['2009', 1211],
 ['2008', 1150],
 ['2007', 1035],
 ['2006', 866],
 ['2005', 738],
 ['2004', 678],
 ['2003', 674],
 ['2002', 659],
 ['2001', 652],
 ['2000', 631],
 ['1999', 645],
 ['1998', 621],
 ['1997', 599],
 ['1996', 571],
 ['1995', 514],
 ['1994', 463],
 ['1993', 413],
 ['1992', 377],
 ['1991', 352],
 ['1990', 314],
 ['1989', 282],
 ['1988', 0],
 ['1987', 0],
 ['1986', 0],
 ['1985', 0],
 ['1984', 0],
 ['1983', 0],
 ['1982', 0],
 ['1981', 0],
 ['1980', 0],
 ['1979', 0],
 ['1978', 0],
 ['1977', 0],
 ['1976', 0],
 ['1975', 0],
 ['1974', 0],
 ['1973', 0],
 ['1972', 0],
 ['1971', 0],
 ['1970', 0],
 ['1969', 0],
 ['1968', 0],
 ['1967', 0],
 ['1966', 0],
 ['1965', 0],
 ['1964', 0],
 ['1963', 0],
 ['1962', 0],
 ['1961', 0],
 ['1960', 0]]

In [19]:
dates = np.arange(2018,1959,-1)
df = pd.DataFrame(index = dates)

for series in oregon:
    if series.get('sector') == 'Residential Sector':

        ts_values = [tuple_[1] for tuple_ in series['data']]
        df = pd.concat([df, pd.Series(data = ts_values,name=series['energy_type'], index=dates)],axis=1)


Natural Gas including Supplemental Gaseous Fuels
Solar energy


ValueError: Length of passed values is 58, index implies 59

In [132]:
df.head()

,Natural Gas including Supplemental Gaseous Fuels,Solar energy,Geothermal,Coal
2018,30969,30969,30969,30969
2017,33987,33987,33987,33987
2016,28570,28570,28570,28570
2015,26962,26962,26962,26962
2014,29185,29185,29185,29185


In [82]:
series.keys()

dict_keys(['_id', 'series_id', 'sector', 'data', 'state', 'units', 'energy_type', 'renewable'])

In [117]:
test_df = pd.concat([or_df, pd.Series(values,name='test',index=dates)],axis=1)

In [118]:
pd.concat([test_df, pd.Series(values, name='test 2',index=dates)],axis=1)

,test,test 2
2018,30969,30969
2017,33987,33987
2016,28570,28570
2015,26962,26962
2014,29185,29185
2013,30832,30832
2012,29451,29451
2011,31034,31034
2010,27461,27461
2009,30504,30504


In [119]:
series['data']

[['2018', 30969],
 ['2017', 33987],
 ['2016', 28570],
 ['2015', 26962],
 ['2014', 29185],
 ['2013', 30832],
 ['2012', 29451],
 ['2011', 31034],
 ['2010', 27461],
 ['2009', 30504],
 ['2008', 31202],
 ['2007', 29960],
 ['2006', 28840],
 ['2005', 28627],
 ['2004', 26438],
 ['2003', 26294],
 ['2002', 28417],
 ['2001', 28690],
 ['2000', 29481],
 ['1999', 30229],
 ['1998', 27285],
 ['1997', 26751],
 ['1996', 26712],
 ['1995', 23417],
 ['1994', 24024],
 ['1993', 25023],
 ['1992', 20318],
 ['1991', 23014],
 ['1990', 20913],
 ['1989', 20980],
 ['1988', 18829],
 ['1987', 17186],
 ['1986', 17214],
 ['1985', 19614],
 ['1984', 17624],
 ['1983', 15764],
 ['1982', 17049],
 ['1981', 15578],
 ['1980', 15872],
 ['1979', 14439],
 ['1978', 12671],
 ['1977', 11317],
 ['1976', 14499],
 ['1975', 16516],
 ['1974', 13991],
 ['1973', 14541],
 ['1972', 15184],
 ['1971', 14173],
 ['1970', 11871],
 ['1969', 10503],
 ['1968', 8299],
 ['1967', 7372],
 ['1966', 7005],
 ['1965', 5983],
 ['1964', 5708],
 ['1963', 4505]